<a href="https://colab.research.google.com/github/ommestriker007/River_timelapse/blob/main/timelapse_of_Subarnarekha_river.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geemap.geemap as geemap
import ee

In [ ]:
ee.Authenticate()
ee.Initialize(project = 'neon-feat-313910')

base = geemap.Map()
# base

In [ ]:
Subarnarekha = ee.FeatureCollection('projects/neon-feat-313910/assets/Subarnarekha')
base.addLayer(Subarnarekha, {}, 'Subarnarekha')
# base

In [ ]:
region = ee.Geometry.Polygon(
    [[[87.0844839, 21.8552299],
      [87.4500633, 21.8062620],
      [87.4617591, 21.4836805],
      [87.0486066, 21.4359486]
      ]],
    None,
    False)

In [ ]:
NDVI = ee.ImageCollection("COPERNICUS/S2_HARMONIZED").select('B4','B3','B2')
base.addLayer(NDVI, {}, 'veg_indices')
# base

In [ ]:
NDVI_river = NDVI.map(lambda img: img.set('doy', ee.Date(img.get('system:time_start')).getRelative('day', 'year')));

In [ ]:
distinctDOY = NDVI_river.filterDate('2016-01-01', '2017-01-01');

In [ ]:
filter = ee.Filter.equals(leftField = 'doy', rightField = 'doy')
join = ee.Join.saveAll('doy_matches')
joincol = ee.ImageCollection(join.apply(distinctDOY, NDVI_river, filter))

In [ ]:
comp = joincol.map(lambda img: ee.ImageCollection.fromImages(img.get('doy_matches')).reduce(ee.Reducer.mean()).set('doy', ee.Number(img.get('doy'))))

In [ ]:
vis_params = {
  'min': 0,
  'max': 9000,
  'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ],
};

In [ ]:
rgbvis = comp.map(lambda img: img.visualize(bands = ['NDVI_mean'], **vis_params).clip(Subarnarekha))

In [ ]:
gifParams = {
  'region': region,
  'dimensions': 800,
  'crs': 'EPSG:4326',
  'framesPerSecond': 10,
  'format':'gif'
};

print(rgbvis.getVideoThumbURL(gifParams));

EEException: Computation timed out.